In [1]:
import sys
import os
import colorsys

import numpy as np
import matplotlib.pyplot as plt
import pycocotools.mask as rletools
from tqdm.notebook import tqdm

from PIL import Image
from multiprocessing import Pool
from mots_tools.mots_common.io import load_sequences, load_seqmap
from functools import partial
from subprocess import call

In [2]:
# adapted from https://github.com/matterport/Mask_RCNN/blob/master/mrcnn/visualize.py
def generate_colors():
  """
  Generate random colors.
  To get visually distinct colors, generate them in HSV space then
  convert to RGB.
  """
  N = 30
  brightness = 0.7
  hsv = [(i / N, 1, brightness) for i in range(N)]
  colors = list(map(lambda c: colorsys.hsv_to_rgb(*c), hsv))
  perm = [15, 13, 25, 12, 19, 8, 22, 24, 29, 17, 28, 20, 2, 27, 11, 26, 21, 4, 3, 18, 9, 5, 14, 1, 16, 0, 23, 7, 6, 10]
  colors = [colors[idx] for idx in perm]
  return colors


# from https://github.com/matterport/Mask_RCNN/blob/master/mrcnn/visualize.py
def apply_mask(image, mask, color, alpha=0.5):
  """Apply the given mask to the image.
  """
  for c in range(3):
    image[:, :, c] = np.where(mask == 1,
                              image[:, :, c] * (1 - alpha) + alpha * color[c],
                              image[:, :, c])
  return image


def process_sequence(seq_id, tracks_folder, img_folder, output_folder, max_frames, draw_boxes=False,
                     create_video=True):
  print("Processing sequence", seq_id)
  os.makedirs(output_folder + "/" + seq_id, exist_ok=True)
  tracks = load_sequences(tracks_folder, [seq_id])[seq_id]
  max_frames_seq = max_frames[seq_id]
  visualize_sequences(seq_id, tracks, max_frames_seq, img_folder, output_folder, draw_boxes, create_video)


def visualize_sequences(seq_id, tracks, max_frames_seq, img_folder, output_folder, draw_boxes=False, create_video=True):
  colors = generate_colors()
  dpi = 100.0
  frames_with_annotations = [frame for frame in tracks.keys() if len(tracks[frame]) > 0]
  img_sizes = next(iter(tracks[frames_with_annotations[0]])).mask["size"]
  for t in range(max_frames_seq + 1):
    print("Processing frame", t)
    filename_t = img_folder + "/" + seq_id + "/%06d" % t
    if os.path.exists(filename_t + ".png"):
      filename_t = filename_t + ".png"
    elif os.path.exists(filename_t + ".jpg"):
      filename_t = filename_t + ".jpg"
    else:
      print("Image file not found for " + filename_t + ".png/.jpg, continuing...")
      continue
    img = np.array(Image.open(filename_t), dtype="float32") / 255
    fig = plt.figure()
    fig.set_size_inches(img_sizes[1] / dpi, img_sizes[0] / dpi, forward=True)
    fig.subplots_adjust(left=0, bottom=0, right=1, top=1, wspace=None, hspace=None)
    ax = fig.subplots()
    ax.set_axis_off()

    if t in tracks:
      for obj in tracks[t]:
        color = colors[obj.track_id % len(colors)]
        if obj.class_id == 1:
          category_name = "Car"
        elif obj.class_id == 2:
          category_name = "Pedestrian"
        else:
          category_name = "Ignore"
          color = (0.7, 0.7, 0.7)
        if obj.class_id == 1 or obj.class_id == 2:  # Don't show boxes or ids for ignore regions
          x, y, w, h = rletools.toBbox(obj.mask)
          if draw_boxes:
            import matplotlib.patches as patches
            rect = patches.Rectangle((x, y), w, h, linewidth=1,
                                     edgecolor=color, facecolor='none', alpha=1.0)
            ax.add_patch(rect)
          category_name += ":" + str(obj.track_id)
          ax.annotate(category_name, (x + 0.5 * w, y + 0.5 * h), color=color, weight='bold',
                      fontsize=7, ha='center', va='center', alpha=1.0)
        binary_mask = rletools.decode(obj.mask)
        apply_mask(img, binary_mask, color)

    ax.imshow(img)
    fig.savefig(output_folder + "/" + seq_id + "/%06d" % t + ".jpg")
    plt.close(fig)
  if create_video:
    os.chdir(output_folder + "/" + seq_id)
    call(["ffmpeg", "-framerate", "10", "-y", "-i", "%06d.jpg", "-c:v", "libx264", "-profile:v", "high", "-crf", "20",
          "-pix_fmt", "yuv420p", "-vf", "pad=\'width=ceil(iw/2)*2:height=ceil(ih/2)*2\'", "output.mp4"])



In [3]:
def process_sequence_2(seq_id, tracks_folder, img_folder, output_folder, max_frames, draw_boxes=False,
                     create_video=True):
    print("Processing sequence", seq_id)
    os.makedirs(output_folder + "/" + seq_id, exist_ok=True)
    tracks = load_sequences(tracks_folder, [seq_id])[seq_id]
    max_frames_seq = max_frames[seq_id]
    visualize_sequences_2(seq_id, tracks, max_frames_seq, img_folder, output_folder, draw_boxes, create_video)


def visualize_sequences_2(seq_id, tracks, max_frames_seq, img_folder, output_folder, draw_boxes=False, create_video=True):
    if not os.path.exists(os.path.join(output_folder, 'sequences', seq_id)):
        os.makedirs(os.path.join(output_folder, 'sequences', seq_id))
    colors = generate_colors()
    for t in range(max_frames_seq + 1):
        # print("Processing frame", t)
        filename_t = img_folder + "/" + seq_id + "/%06d" % t
        if os.path.exists(filename_t + ".png"):
            filename_t = filename_t + ".png"
        elif os.path.exists(filename_t + ".jpg"):
            filename_t = filename_t + ".jpg"
        else:
            print("Image file not found for " + filename_t + ".png/.jpg, continuing...")
            continue
        img = np.array(Image.open(filename_t), dtype="float32") / 255
        # fig = plt.figure()
        # fig.set_size_inches(img_sizes[1] / dpi, img_sizes[0] / dpi, forward=True)
        # fig.subplots_adjust(left=0, bottom=0, right=1, top=1, wspace=None, hspace=None)
        # ax = fig.subplots()
        # ax.set_axis_off()

        if t in tracks:
            for obj in tracks[t]:
                if not os.path.exists(os.path.join(output_folder, 'sequences', seq_id, str(obj.track_id))):
                    os.makedirs(os.path.join(output_folder, 'sequences', seq_id, str(obj.track_id)))
                color = colors[obj.track_id % len(colors)]
                if obj.class_id == 1:
                    category_name = "Car"
                elif obj.class_id == 2:
                    category_name = "Pedestrian"
                else:
                    category_name = "Ignore"
                    color = (0.7, 0.7, 0.7)
                if obj.class_id == 1 or obj.class_id == 2:  # Don't show boxes or ids for ignore regions
                    x, y, w, h = map(int, rletools.toBbox(obj.mask))
                    fig = plt.figure()
                    ax = fig.subplots()
                    ax.set_axis_off()
                    ax.imshow(img[y:y+h,x:x+w])
                    fig.savefig(os.path.join(output_folder, 'sequences', seq_id, str(obj.track_id), "%06d" % t + ".jpg"))
                # if obj.class_id == 1 or obj.class_id == 2:  # Don't show boxes or ids for ignore regions
                #     x, y, w, h = rletools.toBbox(obj.mask)
                #     if draw_boxes:
                #         import matplotlib.patches as patches
                #         rect = patches.Rectangle((x, y), w, h, linewidth=1,
                #                                  edgecolor=color, facecolor='none', alpha=1.0)
                #         ax.add_patch(rect)
                #     category_name += ":" + str(obj.track_id)
                #     ax.annotate(category_name, (x + 0.5 * w, y + 0.5 * h), color=color, weight='bold',
                #               fontsize=7, ha='center', va='center', alpha=1.0)
                # binary_mask = rletools.decode(obj.mask)
                # apply_mask(img, binary_mask, color)
                plt.close(fig)

        # ax.imshow(img)
        # fig.savefig(output_folder + "/" + seq_id + "/%06d" % t + ".jpg")
        # plt.close(fig)
    # if create_video:
    #     os.chdir(output_folder + "/" + seq_id)
    #     call(["ffmpeg", "-framerate", "10", "-y", "-i", "%06d.jpg", "-c:v", "libx264", "-profile:v", "high", "-crf", "20",
    #           "-pix_fmt", "yuv420p", "-vf", "pad=\'width=ceil(iw/2)*2:height=ceil(ih/2)*2\'", "output.mp4"])

In [ ]:
tracks_folder = './kitti-mots/instances_txt'
img_folder = './kitti-mots/data_tracking_image_2/training/image_02/'
output_folder = './outputs'
seqmap_filename = './kitti-mots/sequence_map.txt'

seqmap, max_frames = load_seqmap(seqmap_filename)
process_sequence_part = partial(process_sequence_2, max_frames=max_frames,
                              tracks_folder=tracks_folder, img_folder=img_folder, output_folder=output_folder)

with Pool(70) as pool:
    pool.map(process_sequence_part, seqmap)
    #for seq in seqmap:
    #  process_sequence_part(seq)

In [2]:
import open_clip
import torch
from PIL import Image

In [5]:
if torch.cuda.is_available():
    device = torch.device('cuda:0')
else:
    device = torch.device('cpu')

In [6]:
device

device(type='cuda', index=0)

In [11]:
model, _, preprocess = open_clip.create_model_and_transforms(
    'ViT-B-32',
    pretrained='laion2b_s34b_b79k',
    device=device,
)

In [12]:
def process_sequence_3(
    seq_id, tracks_folder, img_folder, output_folder,
    max_frames, draw_boxes=False, create_video=True,
):
    print("Processing sequence", seq_id)
    os.makedirs(output_folder + "/" + seq_id, exist_ok=True)
    tracks = load_sequences(tracks_folder, [seq_id])[seq_id]
    max_frames_seq = max_frames[seq_id]

    assert os.path.exists(os.path.join(output_folder, 'sequences', seq_id))
    if not os.path.exists(os.path.join(output_folder, 'features', seq_id)):
        os.makedirs(os.path.join(output_folder, 'features', seq_id))
    for t in tqdm(range(max_frames_seq + 1)):
        # print("Processing frame", t)

        if t in tracks:
            for obj in tracks[t]:
                assert os.path.exists(os.path.join(output_folder, 'sequences', seq_id, str(obj.track_id)))
                if not os.path.exists(os.path.join(output_folder, 'features', seq_id, str(obj.track_id))):
                    os.makedirs(os.path.join(output_folder, 'features', seq_id, str(obj.track_id)))
                if obj.class_id == 1 or obj.class_id == 2:  # Don't show boxes or ids for ignore regions
                    img = Image.open(os.path.join(output_folder, 'sequences', seq_id, str(obj.track_id), "%06d" % t + ".jpg"))
                    image = preprocess(img).unsqueeze(0)
                    with torch.no_grad(), torch.cuda.amp.autocast():
                        image_features = model.encode_image(image.to(device))
                        torch.save(
                            image_features,
                            os.path.join(
                                output_folder,
                                'features',
                                seq_id,
                                str(obj.track_id),
                                "%06d" % t + ".pt",
                            )
                        )


In [13]:
tracks_folder = './kitti-mots/instances_txt'
img_folder = './kitti-mots/data_tracking_image_2/training/image_02/'
output_folder = './outputs'
seqmap_filename = './kitti-mots/sequence_map.txt'

seqmap, max_frames = load_seqmap(seqmap_filename)
process_sequence_part = partial(process_sequence_3, max_frames=max_frames,
                              tracks_folder=tracks_folder, img_folder=img_folder, output_folder=output_folder)

print('GPU:', torch.cuda.is_available())

for seq in seqmap:
    process_sequence_part(seq)

Loading seqmap...
GPU: True
Processing sequence 0000
Loading sequence 0000


100%|██████████| 154/154 [00:05<00:00, 30.07it/s]


Processing sequence 0020
Loading sequence 0020


100%|██████████| 837/837 [01:10<00:00, 11.95it/s]


In [3]:
import pinecone

In [4]:
pinecone.init(api_key=os.environ["PINECONE_API"], environment="gcp-starter")

In [5]:
pinecone.create_index

<function pinecone.manage.create_index(name: str, dimension: int, timeout: int = None, index_type: str = 'approximated', metric: str = 'cosine', replicas: int = 1, shards: int = 1, pods: int = 1, pod_type: str = 'p1', index_config: dict = None, metadata_config: dict = None, source_collection: str = '')>

In [7]:
pinecone.create_index("tracks", dimension=512, metric="euclidean")
# pinecone.describe_index("tracks")

In [8]:
pinecone.describe_index('tracks')

IndexDescription(name='tracks', metric='euclidean', replicas=1, dimension=512.0, shards=1, pods=1, pod_type='starter', status={'ready': True, 'state': 'Ready'}, metadata_config=None, source_collection='')

In [9]:
index = pinecone.Index('tracks')

In [10]:
def flush(vectors, force=False):
    if len(vectors) > 20 or (force and len(vectors) > 0):
        index.upsert(
            vectors=[
                {"id": f"{t}-{tid}", "values": vector}
                for tid, t, vector in vectors
            ],
            namespace="images"
        )
        return []
    return vectors


def process_sequence_4(
    seq_id, tracks_folder, img_folder, output_folder,
    max_frames, draw_boxes=False, create_video=True,
):
    print("Processing sequence", seq_id)
    os.makedirs(output_folder + "/" + seq_id, exist_ok=True)
    tracks = load_sequences(tracks_folder, [seq_id])[seq_id]
    max_frames_seq = max_frames[seq_id]

    vectors = []
    assert os.path.exists(os.path.join(output_folder, 'features', seq_id))
    for t in tqdm(range(max_frames_seq + 1)):
        # print("Processing frame", t)

        if t in tracks:
            for obj in tracks[t]:
                assert os.path.exists(os.path.join(output_folder, 'features', seq_id, str(obj.track_id)))
                if obj.class_id == 1 or obj.class_id == 2:  # Don't show boxes or ids for ignore regions
                    with torch.no_grad(), torch.cuda.amp.autocast():
                        features = torch.load(
                            os.path.join(
                                output_folder,
                                'features',
                                seq_id,
                                str(obj.track_id),
                                "%06d" % t + ".pt",
                            )
                        )
                        vectors.append((obj.track_id, t, features.reshape((-1,)).detach().cpu().tolist()))
                        vectors = flush(vectors)
    
    flush(vectors, force=True)

In [11]:
tracks_folder = '/code/kitti-mots/instances_txt'
img_folder = '/code/kitti-mots/data_tracking_image_2/training/image_02/'
output_folder = '/code/outputs'
seqmap_filename = '/code/kitti-mots/sequence_map.txt'

seqmap, max_frames = load_seqmap(seqmap_filename)
process_sequence_part = partial(process_sequence_4, max_frames=max_frames,
                              tracks_folder=tracks_folder, img_folder=img_folder, output_folder=output_folder)

print('GPU:', torch.cuda.is_available())

for seq in seqmap:
    process_sequence_part(seq)

Loading seqmap...
GPU: True
Processing sequence 0000
Loading sequence 0000


  0%|          | 0/154 [00:00<?, ?it/s]

Processing sequence 0020
Loading sequence 0020


  0%|          | 0/837 [00:00<?, ?it/s]

In [12]:
vec = torch.load('/code/outputs/features/0020/1012/000184.pt')

In [16]:
_vec = vec.reshape((-1,)).cpu().tolist()

In [18]:
index.query(
  namespace="images",
  vector=_vec,
  top_k=20,
  include_value=False
)

{'matches': [{'id': '184-1012', 'score': 0.0, 'values': []},
             {'id': '183-1012', 'score': 8.79399109, 'values': []},
             {'id': '182-1012', 'score': 10.2688141, 'values': []},
             {'id': '187-1012', 'score': 11.3680267, 'values': []},
             {'id': '186-1012', 'score': 11.8865814, 'values': []},
             {'id': '185-1012', 'score': 14.087265, 'values': []},
             {'id': '188-1012', 'score': 15.1789856, 'values': []},
             {'id': '181-1012', 'score': 15.5930939, 'values': []},
             {'id': '548-1088', 'score': 16.9915619, 'values': []},
             {'id': '193-1012', 'score': 17.2358093, 'values': []},
             {'id': '322-1012', 'score': 18.2238, 'values': []},
             {'id': '600-1012', 'score': 18.4443054, 'values': []},
             {'id': '595-1012', 'score': 18.5437927, 'values': []},
             {'id': '190-1012', 'score': 19.0632782, 'values': []},
             {'id': '596-1012', 'score': 19.9517822, 'value